# Name:

# Matrix Tiling and Cache

In this notebook, we will use the fiber-tree emulator to display the behavior of various matrix multiplications for dense data. Because the data is assumed to be dense we use the position-based operators on the premise that for dense data the position and coordinate are the same.

In Einsum notation we will be computing the following same as the Part 1:

$$
C_{m,n} = \sum_k A_{m,k} \times B_{k,n}
$$

In particular, we will observe how __matrix tiling__ changes the memory access pattern. Fibertree will provide an intuitive visualization of matrix tiling. 

In [ ]:
# Run boilerplate code to set up environment
from copy import deepcopy
import numpy as np
import random

from cache import Cache, CacheAssoc

%run ./prelude.py --style=uncompressed --animation=movie

## Matrix Multiplication Input Selections

Tiling is expressed by splitting the shape of each rank into two factors, e.g., M1 and M0, where the product is the full shape of the dimension, e.g., M = M1 x M0.

In [ ]:
# Initial values
M1 = 2
M0 = 2
K1 = 2
K0 = 2
N1 = 2
N0 = 2

density = [1.0]
seed = 10

enable_log = False

def set_params(**kwargs):
    global enable_log
    
    for variable, value in kwargs.items():
        globals()[variable] = value

    enable_log = (kwargs["log"] == 'enable')


def logger(arg):
    if enable_log:
        print(arg)

controls = interactive(set_params,
                       M1=widgets.IntSlider(min=1, max=4, step=1, value=M1),
                       M0=widgets.IntSlider(min=1, max=4, step=1, value=M0),
                       K1=widgets.IntSlider(min=1, max=4, step=1, value=K1),
                       K0=widgets.IntSlider(min=1, max=4, step=1, value=K0),
                       N1=widgets.IntSlider(min=1, max=4, step=1, value=N1),
                       N0=widgets.IntSlider(min=1, max=4, step=1, value=N0),
                       seed=widgets.IntSlider(min=0, max=100, step=1, value=seed),
                       log=['disable', 'enable'])

display(controls)
createRunallButton()

## Create Input Tensors

Given shapes selected above create and display the filter weights (**f**) and input activations (**i**) and a reference output (**o_verify**)

In [ ]:
M = M1*M0
K = K1*K0
N = N1*N0

a_MK_raw = []
for m in range(M):
    a_MK_raw.append([random.randint(1, 9) for i in range(K)])
                 
b_KN_raw = []
for k in range(K):
    b_KN_raw.append([random.randint(1, 9) for i in range(N)])

a_MK = Tensor.fromUncompressed(["M", "K"], a_MK_raw)
b_KN = Tensor.fromUncompressed(["K", "N"], b_KN_raw)

a_MK.setName("A_MK").setColor("blue")
b_KN.setName("B_KN").setColor("green")

print("Input A")
displayTensor(a_MK)
                    
print("Input B")
displayTensor(b_KN)

z_verify = None

def create_z():
    """
    Create a fully populated z tensor
    """
    z = Tensor(rank_ids=["M", "N"])
    z.setName("Z")
    z.setMutable(True)

    z_m = z.getRoot()
    #
    # Hack to fill in all the entries in z
    # This allows us to pretend the tensor is dense
    #
    n_fiber = Fiber(coords=range(N), initial=1)
    m_fiber = Fiber(coords=range(M), initial=1)

    for m, (z_n, _) in z_m << m_fiber:
        for n, (z_ref, _) in z_n << n_fiber:
            z_ref <<= 0
            
    return z

## Utility function for addressing

In [ ]:
def getAddress(tensor, x, y):
    return (x*tensor.getShape()[1]+y)

## Matrix Multiply

We are visualizing matrix multiplication we examined in Question 2. You will observe same cache statistics.

In [ ]:
cache_a = Cache(log_size=3, words_per_line=1)
cache_b = Cache(log_size=3, words_per_line=1)
cache_c = Cache(log_size=3, words_per_line=1)

In [ ]:
print("Matrix Multiply")

z_MN = create_z()

print("Output - before")
displayTensor(z_MN)

z = z_MN.getRoot()
a = a_MK.getRoot()
b = b_KN.getRoot()

canvas = createCanvas(a_MK, b_KN, z_MN)
for m in range(M):
    a_tile = [ (m, kt) for kt in range(K)]
    for n in range(N):
        logger(f"Processing Z({m},{n}) = {z[m][n].payload}")
        b_tile = [ (kt, n) for kt in range(K)]
        z_tile = (m, n)
        for k in range(K):
            logger(f"Processing A({m},{k}) = {a[m][k].payload}")
            logger(f"Processing B({k},{n}) = {b[k][n].payload}")
            
            cache_a.load(getAddress(a, m, k))
            cache_b.load(getAddress(b, k, n))
              
            cache_c.load(getAddress(z, m, n))
            z[m][n] += a[m][k] * b[k][n]
            addActivity(canvas, a_tile, b_tile, z_tile, worker="W")
            addFrame(canvas, (m,k), (k,n), (m,n))
        
            cache_c.store(getAddress(z, m, n))

print("Output - after")
displayTensor(z)

displayCanvas(canvas)

if z_verify is None:
    z_verify = deepcopy(z)

# Print cache statistics
print("-------Cache A--------")
cache_a.print_stats()
print("-------Cache B--------")
cache_b.print_stats()
print("-------Cache C--------")
cache_c.print_stats()

## Tiled Matrix Multiply

### Question 8

Here we are simulating __tiled__ matrix multiply. Caches are same as above. You do not need to code anything here. Why do cache statistics change (espeicllay for Cache B) when we use tiling?


_your-answer-here_



In [ ]:
cache_a = Cache(log_size=3, words_per_line=1)
cache_b = Cache(log_size=3, words_per_line=1)
cache_c = Cache(log_size=3, words_per_line=1)

In [ ]:
print("Matrix Multiply")

z_MN = create_z()

print("Output - before")
displayTensor(z_MN)

z = z_MN.getRoot()
a = a_MK.getRoot()
b = b_KN.getRoot()

canvas = createCanvas(a_MK, b_KN, z_MN)

for k1 in range(K1):
    for m1 in range(M1):
        logger(f"Processing Z({m},{n}) = {z[m][n].payload}")
        for n1 in range(N1):
            a_tile = [ (m1*M0+mt, k1*K0+kt) for mt in range(M0)for kt in range(K0)]
            b_tile = [ (k1*K0+kt, n1*N0+nt) for kt in range(K0)for nt in range(N0)]
            z_tile = [ (m1*M0+mt, n1*N0+nt) for mt in range(M0)for nt in range(N0)]

            for m0 in range(M0):
                for n0 in range(N0):
                    for k0 in range(K0):
                        m = m1*M0+m0
                        n = n1*N0+n0
                        k = k1*K0+k0
                        
                        logger(f"Processing A({m},{k}) = {a[m][k].payload}")
                        logger(f"Processing B({k},{n}) = {b[k][n].payload}")
                        
                        cache_a.load(getAddress(a, m, k))
                        cache_b.load(getAddress(b, k, n))
                        
                        cache_c.load(getAddress(z, m, n))
                        
                        z[m][n] += a[m][k] * b[k][n]
                        
                        addActivity(canvas, a_tile, b_tile, z_tile, worker="W")
                        addFrame(canvas, (m,k), (k,n), (m,n))
                        
                        cache_c.store(getAddress(z, m, n))

print("Output - after")
displayTensor(z)

displayCanvas(canvas)

if z_verify is None:
    print("Result not verified")
else:
    assert z == z_verify
    
# Print cache statistics
print("-------Cache A--------")
cache_a.print_stats()
print("-------Cache B--------")
cache_b.print_stats()
print("-------Cache C--------")
cache_c.print_stats()